In [22]:
from src.dataloader import DataLoader
from src.solver import solve_lp

dataloader = DataLoader()
variables, A, B, L = dataloader.load()

In [23]:
i = 36
result = solve_lp((i, variables[i]), L, A, B)

result

(36,
 {'status': 'primal infeasible',
  'x': None,
  's': None,
  'y': None,
  'z': None,
  'primal objective': None,
  'dual objective': None,
  'gap': None,
  'relative gap': None,
  'primal infeasibility': None,
  'dual infeasibility': None,
  'primal slack': None,
  'dual slack': None,
  'residual as primal infeasibility certificate': None,
  'residual as dual infeasibility certificate': None,
  'elapsed': 0.10120534896850586,
  'prob_num': 36})

In [6]:
import multiprocessing 

queue = multiprocessing.Queue()

def task_lp(variables_batch, batch_idx, L, A, B):
    results = []
    for i, row_variable in zip(batch_idx, variables_batch):
        results.append(solve_lp((i, row_variable), L, A, B))
    
    return results

def main():
    num_probs = 120
    num_workers = 3
    batch_size = 30
    
    variables, A, B, L = dataloader.load()
    
    # with multiprocessing.Pool(processes=num_workers) as pool:
    for i in range(0, num_probs, batch_size * num_workers):
        
        processes = []
        for j in range(num_workers):
            from_idx = i + j * batch_size
            to_idx =  i + (j +1) * batch_size 
            
            if from_idx > num_probs - 1:
                continue
            
            elif to_idx > num_probs - 1:
                to_idx = num_probs
                
            batch_idx = range(from_idx, to_idx)

            process = multiprocessing.Process(target = task_lp, args = (variables[from_idx:to_idx], batch_idx, L, A, B))
            processes.append(process)
            
        for process in processes:
            process.start()
            
        for process in processes:
            process.join()

In [11]:
with multiprocessing.Pool(processes=2) as pool:
    
    async_results = [pool.apply_async(task_lp, args=(variables[i:i*10], range(i, i*10), L, A, B)) for i in range(2)]
    # for process in processes:
    #     process.start()

    # for process in processes:
    #     process.join()

    
    # task_lp(variables[0:10], range(0, 10), L, A, B)

In [13]:
async_results[0].get()